# EDA探索性数据分析

In [ ]:
import pandas as pd
df_attr = pd.read_csv('../data/Antai_hackathon_attr.csv')
df_train = pd.read_csv('../data/Antai_hackathon_train.csv')
df_test = pd.read_csv('../data/dianshang_test.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

### 数据检查 -- 能用吗？

##### 检查缺失值           -- 三组数据无缺失值。

- 商品数据

In [ ]:
df_attr.isnull().sum()

- 训练数据

In [ ]:
df_train.isnull().sum()

- 测试数据

In [ ]:
df_test.isnull().sum()

##### 检查重复值  -- 无重复

- 商品数据

In [ ]:
df_attr.duplicated().sum()

- 训练数据

In [ ]:
df_train.duplicated().sum()

- 测试数据

In [ ]:
df_test.duplicated().sum()

##### 检查异常值 数据量 记录数  -- 无异常

- 商品数据 记录数1924269条

In [ ]:
df_attr.describe()

- 训练数据 记录数6989817条

In [28]:
df_train.describe()

,buyer_admin_id,item_id,irank
count,6989817.00,6989817.00,6989817.00
mean,3793426.60,3792787.50,29.33
std,2190326.97,2189485.77,302.32
min,1.00,1.00,1.00
25%,1896300.00,1896413.00,4.00
50%,3795519.00,3793176.00,8.00
75%,5689123.00,5689069.00,13.00
max,7585728.00,7585750.00,11766.00


- 测试数据 记录数140380条

In [ ]:
df_test.describe()

- 检查训练数据中的用户和商品数量

In [29]:
train_buyers = df_train['buyer_admin_id'].nunique()
train_items = df_train['item_id'].nunique()
print(f"训练数据中包含的用户数量为: {train_buyers}")
print(f"训练数据中包含的商品数量为: {train_items}")

训练数据中包含的用户数量为: 483117
训练数据中包含的商品数量为: 1852506


- 检查测试数据中的用户和商品数量

In [30]:
test_buyers = df_test['buyer_admin_id'].nunique()
test_items = df_test['item_id'].nunique()
print(f"测试数据中包含的用户数量为: {test_buyers}")
print(f"测试数据中包含的商品数量为: {test_items}")

测试数据中包含的用户数量为: 10576
测试数据中包含的商品数量为: 91513


- 检查商品属性表中的商品数量

In [ ]:
attr_items = df_attr['item_id'].nunique()
print(f"商品属性表中包含的商品数量为: {attr_items}")